# CapFloor Fundamentals

This notebook demonstrates how to access and use the **Cap & Floor** functionalities within the **LSEG Financial Analytics SDK**. 

Cap and Floor instruments are interest rate derivatives that provide protection against adverse interest rate movements. Caps protect against rising rates while floors protect against falling rates, making them essential tools for risk management and structured products in fixed income markets.

**You will be able to:**
- Define Cap/Floor instruments with basic parameters
- Configure pricing parameters
- Evaluate Cap/Floor Analytics

## Imports

Import the following necessary modules:

- `lseg_analytics.pricing.instruments.cap_floor` - for CapFloor instruments definitions and analytics

This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments import cap_floor as cf

import pandas as pd
import json
import datetime as dt
from IPython.display import display

## Data Preparation

To define a Cap/Floor instrument you need to follow a structured 3-step process:

1. **CapFloor Definition** - Specify basic cap/floor parameters (strike, dates, notional, index)
2. **CapFloor Instrument Definition** - Create the instrument object
3. **Pricing Preferences** - Configure pricing parameters, optional


## 

In [2]:
# 1. Define the cap instrument
cap_definition = cf.IPACapFloorDefinition(
    buy_sell = cf.IPABuySellEnum.BUY.value,                                      # Buy cap protection
    cap_strike_percent = 2.0,                                                 # 2% strike rate
    start_date = dt.datetime.strptime("2025-01-01", "%Y-%m-%d"),              # Start date
    end_date = dt.datetime.strptime("2030-01-01", "%Y-%m-%d"),                # Maturity date
    notional_amount = 1_000_000,                                              # $1M notional
    notional_ccy = "USD",                                                     # USD currency
    index_name = "SOFR",                                                      # SOFR index
    index_tenor = "ON",
    interest_payment_frequency = cf.IndexResetFrequencyEnum.QUARTERLY.value,     # Quarterly payments
)

cap_instrument = cf.IPACapFloorDefinitionInstrument(definition = cap_definition)
print("Instrument definition created")

# 2. Configure pricing parameters
pricing_params = cf.CapFloorPricingParameters(
    valuation_date = dt.datetime.strptime("2025-07-18", "%Y-%m-%d"),                
)
print("Pricing parameters configured")

Instrument definition created
Pricing parameters configured


## Request Execution

In [3]:
#  Execute the calculation using the price() function
# The 'definitions' parameter accepts a list of instruments definitions for batch processing

response = cf.price(
    definitions = [cap_instrument],
    pricing_preferences = pricing_params,
    fields = "MarketValueInDealCcy, marketValueInReportCcy, premiumPercent, premiumBp"  # optional
)

print("Pricing execution completed")


Pricing execution completed


## Results Display

#### Key Sections in the `response` JSON

 - **definitions**: Instrument setup (e.g., strike, dates, notional, index), it's IPACapFloorDefinition that we used. 

 - **pricingPreferences**: Valuation date, price side, currency, skip first caplet.

 - **analytics**:
   - **tabularData**: `data`, `headers`, `statuses`
   - **description**: Instrument summary IPACapFloorDefinition and also the default fields not specified in the IPACapFloorDefinition, but used by default in the calculation
   - **valuation**: customizable list of analytics (`marketValueInDealCcy`, `premiumPercent`, `premiumBp`)
   - **error**: root cause of the issue, if any


In [4]:
# Access the valuation object
valuation = response.data.analytics[0].valuation
print(json.dumps(valuation.as_dict(), indent = 4))

{
    "marketValueInDealCcy": 70192.0568261817,
    "premiumPercent": 7.01920568261817,
    "premiumBp": 701.920568261817
}


In [5]:
# Convert to DataFrame
df_valuation = pd.DataFrame(list(valuation.items()), columns=["Fields", "Value"])
# Display the DataFrame
display(df_valuation)


,Fields,Value
0,marketValueInDealCcy,70192.056826
1,premiumPercent,7.019206
2,premiumBp,701.920568


## Appendix: Complete Functions Reference

The following is a complete list of all classes, enums, and functions available after importing from `lseg_analytics.pricing.instruments.cap_floor`:

**Core Classes:**
- `IPACapFloorDefinition` - Define cap/floor instrument specifications
- `IPACapFloorDefinitionInstrument` - Instrument container for pricing operations
- `CapFloorPricingParameters` - Configure pricing parameters and market data settings

**Response Classes:**
- `CapFloorPricingResponse` - Main response container
- `CapFloorValuation` - Individual instrument valuation results
- `CapFloorAnalytics` - Detailed analytics and calculations

**Configuration Enums:**
- `IPABuySellEnum` - Buy/Sell direction (BUY, SELL)
- `InterestCalculationMethodEnum` - Interest calculation methods (DCB_ACTUAL_360, etc.)
- `IndexResetFrequencyEnum` - Reset frequencies (QUARTERLY, MONTHLY, etc.)
- `PaymentBusinessDayConventionEnum` - Business day conventions (MODIFIED_FOLLOWING, etc.)
- `PaymentRollConventionEnum` - Roll conventions (LAST, EOM, etc.)
- `PriceSideEnum` - Market price sides (BID, ASK, MID)

**Market Data Classes:**
- `InterestRateCurve` - Interest rate curve definitions
- `VolatilitySurface` - Volatility surface configurations
- `MarketDataSet` - Complete market data specifications

**Main Functions:**
- `price()` - Execute cap/floor pricing calculations
- `calculate_greeks()` - Calculate sensitivity measures
- `generate_cashflows()` - Generate detailed cashflow schedules